# 📘 Notebook 1: Tenant & Common Masters Setup

---

## 🎯 Purpose

This notebook handles the foundational setup for your DIGIT CRS system:

- **Phase 0:** Authentication with DIGIT Gateway
- **Phase 1:** Tenant & Branding Upload
- **Phase 3:** Common Masters (Departments, Designations, Complaint Types)

---

## ✅ Prerequisites Checklist

Before starting, ensure you have:

- [ ] **Gateway URL** - Your CRS domain (e.g., `https://unified-dev.digit.org`)
- [ ] **Admin Credentials** - Username and password with MDMS_ADMIN role
- [ ] **Root Tenant ID** - State/root tenant code (e.g., `pg`, `statea`)
- [ ] **Internet Connection** - Stable connection required
- [ ] **Excel Templates** - Will be downloaded during the process

---


## 📋 What You Will Create

### Phase 1 - Tenant & Branding:
- Tenant information (City/State)
- Branding configuration (logos, colors, theme)
- Basic localization keys

### Phase 3 - Common Masters:
- **Departments** with auto-generated codes (DEPT_1, DEPT_2...)
- **Designations** with auto-generated codes (DESIG_01, DESIG_02...)
- **Complaint Types** with service codes and SLA
- Automatic localization for all masters

---

## 🚨 Important Notes

### DO ✅
- Run cells **in sequence** from top to bottom
- Read instructions carefully before each step
- Save Excel files before uploading
- Keep original templates as backup

### DON'T ❌
- Skip authentication (Phase 0)
- Change column names in templates
- Run cells out of order
- Modify system-generated codes

---

## 🔄 How to Use This Notebook

### Running Cells
- **Option 1:** Click on a cell and press `Shift + Enter`
- **Option 2:** Click the ▶️ Run button in the toolbar
- **Option 3:** Use menu: `Cell` → `Run Cells`

### Cell Types
- **Markdown cells** (like this one) - Instructions and documentation
- **Code cells** - Python code to execute

### Status Indicators
- `[*]` - Cell is running
- `[1]` - Cell completed (number shows execution order)
- No bracket - Cell not yet run

---

## 📞 Getting Help

If you encounter issues:
1. Check error messages carefully
2. Review the "Common Errors" sections
3. Verify your credentials and URLs

---

**Ready? Let's begin! 👇**

In [ ]:
!pip install notebook jupyterlab ipykernel ipywidgets pandas openpyxl xlsxwriter tqdm requests python-dotenv sqlalchemy psycopg2-binary plotly
!jupyter nbextension enable --py widgetsnbextension


In [ ]:
import os
from IPython.display import display, HTML

# Templates are in parent directory (pgr_dataLoader/templates)
templates_path = os.path.join("..", "templates")

template_files = [
    ("Tenant And Branding Master.xlsx", "PHASE 1: Tenant Info (4 fields) + Branding"),
    ("Common and Complaint Master.xlsx", "PHASE 3: Departments, Designations & Complaint Types"),
]

html_links = []
for i, (filename, description) in enumerate(template_files, 1):
    filepath = os.path.join(templates_path, filename)
    if os.path.exists(filepath):
        html_links.append(
            f'''
            <div style="margin: 8px 0;">
                <b>{i}.</b>
                <a href="{filepath}" download="{filename}"
                   style="font-size: 14px; color: #0066cc; text-decoration: none;">
                    {filename}
                </a>
                <span style="color: #666;"> - {description}</span>
            </div>
            '''
        )
    else:
        html_links.append(
            f'''
            <div style="margin: 8px 0;">
                <b>{i}.</b> {filename}
                <span style="color: red;">(Not found at: {filepath})</span>
            </div>
            '''
        )

html_content = f"""
<div style="border: 1px solid #ddd; padding: 15px; border-radius: 6px; background-color: #f9f9f9;">
    <h3 style="margin-top: 0;">📁 MDMS Automation Templates</h3>

    {''.join(html_links)}

    <hr style="margin: 18px 0;">

    <p><b>🧭 3-Phase Workflow:</b></p>
    <ol>
        <li><b>PHASE 1:</b> Tenant + Branding (Tenant And Branding Master.xlsx)</li>
        <li><b>PHASE 3:</b> Departments, Designations & Complaint Types</li>
    </ol>

    <p><b>✨ Features:</b></p>
    <ul>
        <li>Only 4 mandatory tenant fields</li>
        <li>ADMIN0/1/2 support for global hierarchy</li>
        <li>Code auto-generation (Dept, Designation, ComplaintTypes)</li>
        <li>Localization auto-generation</li>
    </ul>
</div>
"""

display(HTML(html_content))

In [ ]:
# Add parent directory to path to import unified_loader
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

# Import required packages
import pandas as pd
import json
import warnings
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, FileLink
from unified_loader import UnifiedExcelReader, APIUploader, clean_nans
import shutil
from dotenv import load_dotenv

warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

# Initialize Global Variables
CONFIG = {}
CONFIG_SET = False
TENANT_FILE = None
COMMON_MASTER_FILE = None
UPLOADED_TENANTS = []
SELECTED_TENANT = None
UPLOADER = None  # Global authenticated uploader instance

# Force reload modules to get latest changes
if 'unified_loader' in sys.modules:
    del sys.modules['unified_loader']
if 'mdms_validator' in sys.modules:
    del sys.modules['mdms_validator']
if 'excel_validator' in sys.modules:
    del sys.modules['excel_validator']

# Clear upload folder
upload_dir = 'upload'
if os.path.exists(upload_dir):
    for filename in os.listdir(upload_dir):
        file_path = os.path.join(upload_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')
    print(f"✅ Cleared all files from {upload_dir}/")

print("✅ Packages loaded successfully!")
print("✅ Modules reloaded with latest changes!")
print()
print("="*70)
print("  ⚙️ AUTHENTICATION REQUIRED")
print("="*70)
print("Please proceed to the next cell to authenticate with the gateway.")

In [ ]:
print("="*70)
print("        🔐 GATEWAY AUTHENTICATION")
print("="*70)
print()

# Authentication widgets
base_url_input = widgets.Text(
    value="",
    description='🌐 Gateway URL:',
    placeholder='e.g., https://unified-dev.digit.org',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

username_input = widgets.Text(
    value="",
    description='👤 Username:',
    placeholder='e.g., DEV_SUPER_ADMIN',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

password_input = widgets.Password(
    value="",
    description='🔑 Password:',
    placeholder='Enter password',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

user_type_input = widgets.Dropdown(
    options=['EMPLOYEE', 'CITIZEN'],
    value="EMPLOYEE",
    description='👔 User Type:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

tenant_id_input = widgets.Text(
    value="",
    description='🏛️ Tenant ID:',
    placeholder='e.g., dev, pg, pb',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

auth_button = widgets.Button(
    description='🔐 Authenticate',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px')
)

auth_output = widgets.Output()

def on_authenticate(b):
    global UPLOADER, CONFIG
    
    with auth_output:
        clear_output()
        
        # Validate inputs
        if not base_url_input.value.strip():
            print("❌ Gateway URL is required")
            return
        
        if not username_input.value.strip():
            print("❌ Username is required")
            return
        
        if not password_input.value.strip():
            print("❌ Password is required")
            return
        
        if not tenant_id_input.value.strip():
            print("❌ Tenant ID is required")
            return
        
        print("🔄 Authenticating with gateway...")
        print(f"   Gateway: {base_url_input.value}")
        print(f"   Username: {username_input.value}")
        print(f"   User Type: {user_type_input.value}")
        print(f"   Tenant ID: {tenant_id_input.value}")
        print()
        
        try:
            # Create APIUploader with authentication
            UPLOADER = APIUploader(
                base_url=base_url_input.value.strip(),
                username=username_input.value.strip(),
                password=password_input.value.strip(),
                user_type=user_type_input.value,
                tenant_id=tenant_id_input.value.strip()
            )
            
            if UPLOADER.authenticated:
                # Save config
                CONFIG = {
                    'base_url': base_url_input.value.strip(),
                    'tenant_id': tenant_id_input.value.strip()
                }
                CONFIG_SET = True
                
                print()
                print("="*70)
                print("  ✅ AUTHENTICATION SUCCESSFUL!")
                print("="*70)
                print(f"\n👤 Logged in as: {UPLOADER.user_info.get('userName', 'Unknown')}")
                print(f"📧 Email: {UPLOADER.user_info.get('emailId', 'N/A')}")
                print(f"🏛️ Tenant: {UPLOADER.user_info.get('tenantId', 'N/A')}")
                print(f"\n🎭 Roles:")
                for role in UPLOADER.user_info.get('roles', []):
                    print(f"   • {role.get('name', 'Unknown')} ({role.get('code', 'N/A')})")
                print()
                print("="*70)
                print("  ➡️ Proceed to PHASE 1: Tenant Setup")
                print("="*70)
            else:
                print("❌ Authentication failed. Please check your credentials.")
        
        except Exception as e:
            print(f"❌ Error during authentication: {str(e)}")

auth_button.on_click(on_authenticate)

# Display UI
auth_ui = widgets.VBox([
    widgets.HTML("<h3>🔐 Step 0: Gateway Authentication</h3>"),
    widgets.HTML("<p style='color: #666;'><i>Authenticate with the eGov gateway to access all services</i></p>"),
    widgets.HTML("<br>"),
    base_url_input,
    widgets.HTML("<p style='font-size: 11px; color: #888;'><i>💡 Gateway URL - same for all services</i></p>"),
    widgets.HTML("<br>"),
    username_input,
    password_input,
    user_type_input,
    tenant_id_input,
    widgets.HTML("<p style='font-size: 11px; color: #ff6600;'><i>⚠️ Credentials are used only for this session and not stored</i></p>"),
    widgets.HTML("<br>"),
    auth_button,
    auth_output
])

display(auth_ui)

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Initialize uploader
uploader = UPLOADER

# Fetch existing tenants
tenants = uploader.search_mdms_data(schema_code='tenant.tenants', tenant=CONFIG['tenant_id'])
tenant_codes = [t["code"] for t in tenants] if tenants else []

# Title
title = widgets.HTML("<h3 style='color:#444;margin-bottom:10px;'>⚙️ Tenant Configuration</h3>")

# Info box
if tenants:
    tenant_list_html = "<br>".join([f"• <b>{t['code']}</b> - {t['name']}" for t in tenants])
    info_box = widgets.HTML(
        f"<div style='background:#e7f3ff;padding:10px;border-radius:5px;'>"
        f"<b>✔ Found {len(tenants)} existing tenant(s) in MDMS:</b><br>{tenant_list_html}</div>"
    )
else:
    info_box = widgets.HTML(
        "<div style='background:#fff3cd;padding:10px;border-radius:5px;'>"
        "<b>⚠ No existing tenants found in MDMS.</b></div>"
    )

# YES/NO question
question = widgets.HTML("<p><b>Do you want to use an existing tenant?</b></p>")

yes_no_buttons = widgets.ToggleButtons(
    options=[('YES - Use Existing', 'yes'), ('NO - Create New', 'no')],
)

# Existing tenant dropdown
existing_dropdown = widgets.Dropdown(
    options=tenant_codes,
    description='Select Tenant:',
    style={'description_width': '120px'},
    layout=widgets.Layout(visibility='hidden')
)

# New tenant UI (each field hidden individually)
new_label = widgets.HTML(
    "<p style='color:#666;font-size:12px;'><i>💡 Fill in details for the new tenant</i></p>",
    layout=widgets.Layout(visibility='hidden')
)

new_tenant_code = widgets.Text(
    placeholder='e.g., pg.cityc, pb.amritsar',
    description='Tenant Code:',
    style={'description_width': '120px'},
    layout=widgets.Layout(visibility='hidden')
)




# Proceed button
proceed_button = widgets.Button(
    description="✅ Proceed",
    button_style='success',
    icon='check',
    layout=widgets.Layout(visibility='hidden')
)

output = widgets.Output()

# ----- VISIBILITY TOGGLE -----
def on_yes_no_change(change):
    choice = change['new']

    if choice == 'yes':
        existing_dropdown.layout.visibility = 'visible'

        # Hide new tenant input fields
        for w in [
            new_tenant_code
        ]:
            w.layout.visibility = 'hidden'

        proceed_button.layout.visibility = 'visible'

    elif choice == 'no':
        existing_dropdown.layout.visibility = 'hidden'

        # Show new tenant input fields
        for w in [
            new_tenant_code
        ]:
            w.layout.visibility = 'visible'

        proceed_button.layout.visibility = 'visible'

yes_no_buttons.observe(on_yes_no_change, names='value')

# ----- PROCEED BUTTON LOGIC -----
def on_proceed_clicked(b):
    with output:
        clear_output()

        # Existing tenant selected
        if yes_no_buttons.value == 'yes':
            selected = existing_dropdown.value
            print(f"✅ Proceeding with existing tenant: {selected}")
            return

        # New tenant flow
        tenant_code = new_tenant_code.value.strip()
        

        # Validation
        if not tenant_code:
            print("❌ Tenant code is required")
            return
      

        print(f"🔄 Creating new tenant: {tenant_code}...\n")

        # ---- CORRECT API CALL ----
        result = uploader.setup_default_data(
            targetTenantId=tenant_code,
            module="tenant",
            schemaCodes=[
              "tenant.citymodule",
              "tenant.tenants"
            ],
            onlySchemas=False
        )

        if result.get("success"):
            print("✅ Tenant created successfully!")
            print(f"➡ Next: Proceed to Phase 2 for tenant: {tenant_code}")
        else:
            print("❌ Failed to create tenant")
            print(result)

proceed_button.on_click(on_proceed_clicked)

# UI Layout
ui = widgets.VBox([
    title,
    info_box,
    question,
    yes_no_buttons,
    existing_dropdown,
    new_label,
    new_tenant_code,
    proceed_button,
    output
])

display(ui)


---

# 🟦 PHASE 1: TENANT/STATE SETUP

## 📋 Overview
In Phase 1, you will set up the foundational tenant and branding configuration for your state/ULB.

## 🎯 What You'll Do:
1. **Enter State Tenant ID** (e.g., `pg` for Punjab, `pb` for Punjab, `mh` for Maharashtra)
2. **Upload Tenant Master Excel** with state and city information
3. **Upload tenant data** - Creates tenant records in MDMS
4. **Upload state branding** - Logo, colors, theme configuration

## 📄 Template Required:
- **Tenant And Branding Master.xlsx**

## ✅ What Gets Created:
- Tenant information in MDMS
- City-level tenant records (e.g., `pg.citya`)
- Branding configuration (logo, theme, colors)
- Basic localization keys
- Admin boundary structure

## ⚠️ Important:
- Make sure the Excel file is in the `templates/` folder
- Tenant ID must be lowercase (e.g., `pg`, not `PG`)
- Complete Phase 1 before moving to Phase 2


In [ ]:
# Configuration Setup and Tenant Master Upload
# Make sure you've run the "Initialize Global Variables" cell first!

print("="*70)
print("        ⚙️  PHASE 1: CONFIGURATION & UPLOAD TENANT MASTER")
print("="*70)
print()

# Configuration widgets
state_tenant_id = widgets.Text(
    value='pg',
    description='🏛️ State Tenant ID:',
    placeholder='e.g., pg, pb, up',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

# File upload widget
tenant_file_upload = widgets.FileUpload(
    accept='.xlsx,.xls',
    multiple=False,
    description='Select File:',
    layout=widgets.Layout(width='70%')
)

# Combined button
upload_btn = widgets.Button(
    description='✅ Save Config & Upload Tenant Master',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px')
)

output = widgets.Output()

def on_upload(b):
    global CONFIG, CONFIG_SET, TENANT_FILE, UPLOADED_TENANTS
    
    with output:
        clear_output()
        
        # Validate configuration
        if not state_tenant_id.value.strip():
            print("❌ State Tenant ID required")
            return
        
        # Validate file selection
        if not tenant_file_upload.value:
            print("❌ Please select Tenant Master Excel file")
            return
        
        # Save configuration
        CONFIG = {
            'base_url': 'https://unified-qa.digit.org',
            'state_tenant_id': state_tenant_id.value.strip().lower()
        }
        CONFIG_SET = True
        
        print("="*70)
        print("  ✅ CONFIGURATION SAVED!")
        print("="*70)
        print(f"\n📊 Configuration:")
        print(f"   • State Tenant: {CONFIG['state_tenant_id']}")
        print("="*70)
        
        # Save file
        os.makedirs('upload', exist_ok=True)
        uploaded_file = tenant_file_upload.value[0]
        content = uploaded_file['content']
        filename = 'Tenant_Master.xlsx'
        
        upload_path = os.path.join('upload', filename)
        with open(upload_path, 'wb') as f:
            f.write(content)
        
        print("\n" + "="*70)
        print("  📋 VALIDATING FILE AGAINST MDMS SCHEMA...")
        print("="*70)
        
        # Validate the file against MDMS schema
        try:
            from mdms_validator import MDMSValidator
            mdms_validator = MDMSValidator(
                base_url=UPLOADER.base_url,
                auth_token=UPLOADER.auth_token,
                user_info=UPLOADER.user_info
            )
            result = mdms_validator.validate_excel_file(
                excel_file=upload_path,
                tenant_id=CONFIG['state_tenant_id'],
                schema_code='tenant.masterschemavalidation'
            )
            
            if result['valid']:
                print("\n✅ MDMS VALIDATION PASSED!")
                print("   All required fields match MDMS schema.")
                TENANT_FILE = upload_path
                
                print("\n" + "="*70)
                print("  ✅ FILE UPLOADED SUCCESSFULLY")
                print("="*70)
                print(f"\n📄 File: {upload_path}")
                print("\n➡️  Proceed to load and upload tenant data")
                print("="*70)
            else:
                print("\n❌ MDMS VALIDATION FAILED!")
                print(f"   Found {len(result['errors'])} error(s):\n")
                
                # Show first 5 errors
                for i, error in enumerate(result['errors'][:5], 1):
                    print(f"   Error {i}:")
                    if 'row' in error:
                        print(f"      Sheet: {error.get('sheet', 'Unknown')}")
                        print(f"      Row: {error['row']}")
                        if 'column' in error:
                            print(f"      Column: {error['column']}")
                        if 'value' in error:
                            print(f"      Value: {error.get('value', 'N/A')}")
                    print(f"      Message: {error['message']}")
                    print()
                
                if len(result['errors']) > 5:
                    print(f"   ... and {len(result['errors']) - 5} more errors")
                
                print("\n⚠️  Please fix the errors in your Excel file and try again.")
                print("="*70)
                TENANT_FILE = None
                CONFIG_SET = False
                
        except Exception as e:
            print(f"\n⚠️  MDMS Validation error: {str(e)}")
            print("   Proceeding without validation...")
            TENANT_FILE = upload_path
            print("\n" + "="*70)
            print("  ✅ FILE UPLOADED SUCCESSFULLY")
            print("="*70)
            print(f"\n📄 File: {upload_path}")
            print("\n➡️  Proceed to load and upload tenant data")
            print("="*70)

upload_btn.on_click(on_upload)

# Display
combined_box = widgets.VBox([
    widgets.HTML("<h3>📋 Step 1.1: Configuration & Upload Tenant Master</h3>"),
    widgets.HTML("<p style='color: #666;'><i>Enter your state tenant ID and upload the Tenant Master Excel file</i></p>"),
    widgets.HTML("<p style='color: #666;'><b>Example:</b> If your state is Punjab, enter <code>pg</code></p>"),
    widgets.HTML("<br>"),
    state_tenant_id,
    widgets.HTML("<p style='font-size: 12px; color: #888;'><i>💡 The Tenant Master Excel will contain cities like pg.citya, pg.cityb under this state tenant</i></p>"),
    widgets.HTML("<br>"),
    widgets.HTML("<p style='color: #666;'><b>Select Tenant Master Excel File:</b></p>"),
    tenant_file_upload,
    widgets.HTML("<p style='font-size: 11px; color: #007bff;'><i>⚡ File will be validated against MDMS schema 'tenant.masterschemavalidation'</i></p>"),
    widgets.HTML("<br>"),
    upload_btn,
    output
])

display(combined_box)

---

## 📤 Step 1.1: Upload Tenant Master Excel

**Instructions:**
1. Click the **Upload** button below
2. Select **Tenant And Branding Master.xlsx** from your computer
3. The file will be uploaded to the `templates/` folder
4. You'll see a confirmation message

**What happens next:**
- System reads tenant and city information from Excel
- Creates tenant records in MDMS
- Sets up basic admin structure

💡 **Tip**: Make sure the Excel file has the correct format with State ID and City details.


In [ ]:
print("="*70)
print("[PHASE 1] LOADING TENANT DATA")
print("="*70)

if not UPLOADER or not UPLOADER.authenticated:
    print("❌ Please authenticate first! Run the Authentication cell (Cell 5).")
    raise SystemExit("Authentication required")


if not TENANT_FILE:
    print("❌ Please upload Tenant Master Excel first!")
else:
    # Initialize reader and uploader
    reader = UnifiedExcelReader(TENANT_FILE)
    uploader = UPLOADER
    
    # Read tenant data using NEW format method
    tenants_data, tenants_localization = reader.read_tenant_info()
    
    print(f"\n[INFO] Loaded {len(tenants_data)} tenant(s) from Excel")
    for tenant in tenants_data:
        print(f"   - {tenant['code']}: {tenant['name']} ({tenant['type']})")
        UPLOADED_TENANTS.append(tenant['code'])
    
    print(f"\n[INFO] Auto-generated {len(tenants_localization)} localization entries")
    
    # Upload Tenants
    result_tenants = uploader.create_mdms_data(
    schema_code='tenant.tenants',
    data_list=clean_nans(tenants_data),
    tenant=CONFIG['state_tenant_id'],
    sheet_name='Tenant Info',  # Exact sheet name from your Excel
    excel_file=TENANT_FILE      # ← Add this
    )
    
    # Upload Localization
    result_loc = uploader.create_localization_messages(
        localization_list=clean_nans(tenants_localization),
        tenant=CONFIG['state_tenant_id'],
        sheet_name='Tenants_Localization'
    )
    print(result_tenants)
    # Summary
    if result_tenants['failed'] == 0:
        print("\n✅ [SUCCESS] Tenants uploaded successfully!")
        print("\n➡️  Proceed to upload State Branding (optional)")
    else:
        print("\n⚠️  [WARNING] Some tenants failed. Check errors/ folder.")

---

## 🎨 Step 1.2: Upload State Branding Configuration

**Instructions:**
1. Run the cell below to process branding data
2. System reads the **State Branding** sheet from Excel
3. Uploads logo, colors, and theme configuration

**What gets configured:**
- State/ULB logo
- Primary and secondary colors
- Login page theme
- Header/footer branding

**Expected Result:**
- ✅ Branding data uploaded to MDMS
- ✅ Logo and theme visible in PGR application

💡 **Tip**: After this step, Phase 1 is complete. Proceed to Phase 2 for boundary setup.


In [ ]:
print("="*70)
print("[PHASE 1] LOADING STATE BRANDING DATA")
print("="*70)

if not UPLOADER or not UPLOADER.authenticated:
    print("❌ Please authenticate first! Run the Authentication cell (Cell 5).")
    raise SystemExit("Authentication required")


if not TENANT_FILE:
    print("❌ Please upload Tenant Master Excel first!")
else:
    # Initialize reader and uploader
    reader = UnifiedExcelReader(TENANT_FILE)
    uploader = UPLOADER
    
    # Read branding data using NEW format method
    branding_data = reader.read_tenant_branding(CONFIG['state_tenant_id'])
    
    if not branding_data:
        print("\n[INFO] No branding data found in 'Tenant Branding Deatils' sheet")
        print("   This is optional - you can skip this step if not needed")
    else:
        print(f"\n[INFO] Loaded {len(branding_data)} branding record(s)")
        for brand in branding_data:
            print(f"   - {brand['code']}")
        
        # Upload Branding to tenant.stateInfo schema
        result_branding = uploader.create_mdms_data(
            schema_code="common-masters.StateInfo",
            data_list=clean_nans(branding_data),
            tenant=CONFIG['state_tenant_id'],
            sheet_name='Tenant Branding Deatils',  # Exact sheet name from your Excel
            excel_file=TENANT_FILE      # ← Add this
        )
        
        # Summary
        if result_branding['failed'] == 0:
            print("\n✅ [SUCCESS] State Branding uploaded successfully!")
            print("\n🎉 PHASE 1 COMPLETED!")
            print("\n➡️  Proceed to PHASE 2: Common Master Data Upload")
        else:
            print("\n⚠️  [WARNING] Some branding records failed. Check errors/ folder.")

---

# 🟪 PHASE 3: COMMON MASTER DATA

## 📋 Overview
In Phase 3, you will configure all common master data and PGR complaint configurations for your tenant.

## 🎯 What You'll Do:
1. **Enter Target Tenant ID** (e.g., `pg.citya` for city-level tenant)
2. **Upload Common Master Excel** with departments, designations, and complaint types
3. **System automatically creates:**
   - Department records in MDMS
   - Designation records in MDMS
   - Complaint types and sub-types
   - SLA configurations
   - Localization entries for all master data

## 📄 Template Required:
- **Common and Complaint Master.xlsx**

## 📊 Sheets in Template:
1. **Department Master** - List all departments (e.g., Public Works, Sanitation)
2. **Designation Master** - List all designations (e.g., Engineer, Supervisor)
3. **Complaint Type Master** - Main complaint categories
4. **Complaint SubType Master** - Sub-categories under each type

## ✅ What Gets Created:
- Departments with auto-generated codes (e.g., `DEPT_1`, `DEPT_2`)
- Designations with auto-generated codes (e.g., `DESIG_01`, `DESIG_02`)
- Complaint types with IDs and SLA hours
- Complaint sub-types linked to parent types
- Localization messages for all entries (en_IN, hi_IN)

## ⚠️ Important:
- Complete Phase 1 and Phase 2 before Phase 3
- Use city-level tenant ID (e.g., `pg.citya`, not just `pg`)
- Departments and Designations created here will be used in Phase 4 (Employee Creation)


---

## 📤 Step 3.1: Upload Common Master Excel

**Instructions:**
1. Enter the **Target Tenant ID** (e.g., `pg.citya`) in the text box
2. Click **Upload** button
3. Select **Common and Complaint Master.xlsx**
4. Click **Set Tenant & Upload** button

**What happens:**
- System reads all sheets (Departments, Designations, Complaint Types, Sub-Types)
- Validates data format
- Stores file for processing in next steps

**Expected Output:**
```
✅ Tenant set: pg.citya
✅ File uploaded: Common and Complaint Master.xlsx
```

💡 **Tip**: Use city-level tenant (e.g., `pg.citya`), not state-level (`pg`).


In [ ]:
print("="*70)
print("        🎯  ENTER TARGET TENANT & UPLOAD COMMON MASTER")
print("="*70)
print()

# Tenant input widget
tenant_input = widgets.Text(
    value='',
    placeholder='e.g., pg, pg.citya, pg.cityb',
    description='🏛️ Tenant ID:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

# File upload widget
common_file_upload = widgets.FileUpload(
    accept='.xlsx,.xls',
    multiple=False,
    description='Select File:',
    layout=widgets.Layout(width='70%')
)

# Combined upload button
upload_btn = widgets.Button(
    description='⬆️ Set Tenant & Upload Common Master',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px')
)

upload_output = widgets.Output()

def on_upload(b):
    global COMMON_MASTER_FILE, SELECTED_TENANT
    
    with upload_output:
        clear_output()
        
        # Validate tenant input
        if not tenant_input.value.strip():
            print("❌ Please enter a tenant ID")
            return
        
        # Validate file selection
        if not common_file_upload.value:
            print("❌ Please select Common Master Excel file")
            return
        
        # Set tenant
        SELECTED_TENANT = tenant_input.value.strip().lower()
        
        # Save file
        os.makedirs('upload', exist_ok=True)
        uploaded_file = common_file_upload.value[0]
        content = uploaded_file['content']
        filename = 'Common_Master.xlsx'
        
        upload_path = os.path.join('upload', filename)
        with open(upload_path, 'wb') as f:
            f.write(content)
        
        print("="*70)
        print("  📋 VALIDATING FILE AGAINST MDMS SCHEMA...")
        print("="*70)
        
        # Validate the file against MDMS schema
        try:
            from mdms_validator import MDMSValidator
            mdms_validator = MDMSValidator(
                base_url=UPLOADER.base_url,
                auth_token=UPLOADER.auth_token,
                user_info=UPLOADER.user_info
            )
            result = mdms_validator.validate_excel_file(
                excel_file=upload_path,
                tenant_id=SELECTED_TENANT,
                schema_code='common.masterschemavalidation'
            )
            
            if result['valid']:
                print("\n✅ MDMS VALIDATION PASSED!")
                print("   All required fields match MDMS schema.")
                COMMON_MASTER_FILE = upload_path
                
                print("\n" + "="*70)
                print("  ✅ FILE UPLOADED SUCCESSFULLY")
                print("="*70)
                print(f"\n🎯 Target Tenant: {SELECTED_TENANT}")
                print(f"\n📄 File: {upload_path}")
                print("\n➡️  Proceed to load and upload departments & designations")
                print("="*70)
            else:
                print("\n❌ MDMS VALIDATION FAILED!")
                print(f"   Found {len(result['errors'])} error(s):\n")
                
                # Show first 5 errors
                for i, error in enumerate(result['errors'][:5], 1):
                    print(f"   Error {i}:")
                    if 'row' in error:
                        print(f"      Sheet: {error.get('sheet', 'Unknown')}")
                        print(f"      Row: {error['row']}")
                        if 'column' in error:
                            print(f"      Column: {error['column']}")
                        if 'value' in error:
                            print(f"      Value: {error.get('value', 'N/A')}")
                    print(f"      Message: {error['message']}")
                    print()
                
                if len(result['errors']) > 5:
                    print(f"   ... and {len(result['errors']) - 5} more errors")
                
                print("\n⚠️  Please fix the errors in your Excel file and try again.")
                print("="*70)
                COMMON_MASTER_FILE = None
                SELECTED_TENANT = None
                
        except Exception as e:
            print(f"\n⚠️  MDMS Validation error: {str(e)}")
            print("   Proceeding without validation...")
            COMMON_MASTER_FILE = upload_path
            print("\n" + "="*70)
            print("  ✅ FILE UPLOADED SUCCESSFULLY")
            print("="*70)
            print(f"\n🎯 Target Tenant: {SELECTED_TENANT}")
            print(f"\n📄 File: {upload_path}")
            print("\n➡️  Proceed to load and upload departments & designations")
            print("="*70)

upload_btn.on_click(on_upload)

# Show uploaded tenants as reference (if available)
if UPLOADED_TENANTS:
    reference_info = widgets.HTML(
        f"<p style='color: #666; background-color: #f0f0f0; padding: 10px; border-radius: 5px;'>"
        f"<b>📋 Reference - Uploaded Tenants from Phase 1:</b><br>"
        f"{', '.join(UPLOADED_TENANTS)}</p>"
    )
else:
    reference_info = widgets.HTML("")

# Display
upload_box = widgets.VBox([
    widgets.HTML("<h3>🎯 Step 2.1: Enter Target Tenant & Upload Common Master</h3>"),
    widgets.HTML("<p style='color: #666;'><i>Enter the tenant ID and upload the Common Master Excel file</i></p>"),
    reference_info,
    widgets.HTML("<br>"),
    tenant_input,
    widgets.HTML("<p style='font-size: 12px; color: #888;'><i>💡 You can upload to any tenant - it doesn't have to be from Phase 1</i></p>"),
    widgets.HTML("<br>"),
    widgets.HTML("<p style='color: #666;'><b>Select Common Master Excel File:</b></p>"),
    common_file_upload,
    widgets.HTML("<p style='font-size: 11px; color: #007bff;'><i>⚡ File will be validated against MDMS schema 'common.masterschemavalidation'</i></p>"),
    widgets.HTML("<br>"),
    upload_btn,
    upload_output
])

display(upload_box)

In [ ]:
print("="*70)
print("[PHASE 2] LOADING DEPARTMENTS & DESIGNATIONS")
print("="*70)

if not UPLOADER or not UPLOADER.authenticated:
    print("❌ Please authenticate first! Run the Authentication cell (Cell 5).")
    raise SystemExit("Authentication required")


if not COMMON_MASTER_FILE:
    print("❌ Please upload Common Master Excel first!")
elif not SELECTED_TENANT:
    print("❌ Please select a tenant first!")
else:
    # Initialize reader
    reader = UnifiedExcelReader(COMMON_MASTER_FILE)
    uploader = UPLOADER
    
    # Read departments and designations (combined sheet)
    # Now also returns dept_name_to_code mapping for complaint types
    departments, designations, dept_loc, desig_loc, dept_name_to_code = reader.read_departments_designations(SELECTED_TENANT, uploader)
    
    # Create reverse mapping (code to name) for error Excel
    dept_code_to_name = {code: name for name, code in dept_name_to_code.items()}
    
    print(f"\n[INFO] Loaded {len(departments)} department(s)")
    for dept in departments:
        print(f"   - {dept['code']}: {dept['name']}")
    
    print(f"\n[INFO] Loaded {len(designations)} designation(s)")
    for desig in designations[:5]:
        print(f"   - {desig['code']}: {desig['name']} [{desig['department']}]")
    if len(designations) > 5:
        print(f"   ... and {len(designations) - 5} more")
    
    print(f"\n[INFO] Auto-generated {len(dept_loc) + len(desig_loc)} localization entries")
    print(f"\n[INFO] Created department name-to-code mapping: {dept_name_to_code}")
    
    # Store reverse mapping for error Excel
    uploader._dept_code_to_name = dept_code_to_name
    
    # Upload Departments (use combined sheet name for errors)
    result_dept = uploader.create_mdms_data(
        schema_code='common-masters.Department',
        data_list=clean_nans(departments),
        tenant=SELECTED_TENANT,
        sheet_name='Department And Desgination Mast',
        excel_file=COMMON_MASTER_FILE
    )
    
    # Upload Designations (use same combined sheet name for errors)
    result_desig = uploader.create_mdms_data(
        schema_code='common-masters.Designation',
        data_list=clean_nans(designations),
        tenant=SELECTED_TENANT,
        sheet_name='Department And Desgination Mast', 
        excel_file=COMMON_MASTER_FILE  
    )
    
    # Upload Localization
    all_loc = dept_loc + desig_loc
    result_loc = uploader.create_localization_messages(
        localization_list=clean_nans(all_loc),
        tenant=SELECTED_TENANT,
        sheet_name='Dept_Desig_Localization'
    )
    
    # Summary
    if result_dept['failed'] == 0 and result_desig['failed'] == 0:
        print("\n✅ [SUCCESS] Departments & Designations uploaded successfully!")
        print("\n➡️  Proceed to upload Complaint Types")
    else:
        print("\n⚠️  [WARNING] Some items failed. Check errors/ folder.")

## Step 2.3: Load and Upload Complaint Types

In [ ]:
print("="*70)
print("[PHASE 2] LOADING COMPLAINT TYPES")
print("="*70)

if not UPLOADER or not UPLOADER.authenticated:
    print("❌ Please authenticate first! Run the Authentication cell (Cell 5).")
    raise SystemExit("Authentication required")


if not COMMON_MASTER_FILE:
    print("❌ Please upload Common Master Excel first!")
elif not SELECTED_TENANT:
    print("❌ Please select a tenant first!")
elif 'dept_name_to_code' not in locals():
    print("❌ Please run 'Load Departments & Designations' cell first!")
else:
    # Initialize reader
    reader = UnifiedExcelReader(COMMON_MASTER_FILE)
    uploader = UPLOADER
    
    # Read complaint types - pass dept_name_to_code mapping so it uses department codes
    complaint_types, ct_loc = reader.read_complaint_types(SELECTED_TENANT, dept_name_to_code)
    
    print(f"\n[INFO] Loaded {len(complaint_types)} complaint type(s)")
    for ct in complaint_types[:10]:
        dept_info = f" [Dept: {ct.get('department', 'N/A')}]" if ct.get('department') else ""
        print(f"   - {ct['serviceCode']}: {ct['name']}{dept_info}")
    if len(complaint_types) > 10:
        print(f"   ... and {len(complaint_types) - 10} more")
    
    print(f"\n[INFO] Auto-generated {len(ct_loc)} localization entries")
    
    # Upload Complaint Types (pass reverse mapping for error Excel)
    uploader._dept_code_to_name = dept_code_to_name
    result_ct = uploader.create_mdms_data(
        schema_code='RAINMAKER-PGR.ServiceDefs',
        data_list=clean_nans(complaint_types),
        tenant=SELECTED_TENANT,
        sheet_name='Complaint Type Master',
          excel_file=COMMON_MASTER_FILE  
    )
    
    # Upload Localization
    result_loc = uploader.create_localization_messages(
        localization_list=clean_nans(ct_loc),
        tenant=SELECTED_TENANT,
        sheet_name='ComplaintTypes_Localization'
    )
    
    # Summary
    if result_ct['failed'] == 0:
        print("\n✅ [SUCCESS] Complaint Types uploaded successfully!")
        print("\n🎉 PHASE 2 COMPLETED!")
    else:
        print("\n⚠️  [WARNING] Some complaint types failed. Check errors/ folder.")

In [ ]:
from IPython.display import display, HTML, FileLink
from datetime import datetime
import os

# Collect results
summary_data = []

if 'result_tenants' in locals():
    summary_data.append({
        'module': 'Tenants',
        'created': result_tenants.get('created', 0),
        'exists': result_tenants.get('exists', 0),
        'failed': result_tenants.get('failed', 0),
        'excel_file': TENANT_FILE if 'TENANT_FILE' in locals() else None,
        'sheet': 'Tenant Info'
    })

if 'result_branding' in locals():
    summary_data.append({
        'module': 'State Branding',
        'created': result_branding.get('created', 0),
        'exists': result_branding.get('exists', 0),
        'failed': result_branding.get('failed', 0),
        'excel_file': TENANT_FILE if 'TENANT_FILE' in locals() else None,
        'sheet': 'Tenant Branding Deatils'
    })

if 'result_dept' in locals():
    summary_data.append({
        'module': 'Departments',
        'created': result_dept.get('created', 0),
        'exists': result_dept.get('exists', 0),
        'failed': result_dept.get('failed', 0),
        'excel_file': COMMON_MASTER_FILE if 'COMMON_MASTER_FILE' in locals() else None,
        'sheet': 'Department And Desgination Mast'
    })

if 'result_desig' in locals():
    summary_data.append({
        'module': 'Designations',
        'created': result_desig.get('created', 0),
        'exists': result_desig.get('exists', 0),
        'failed': result_desig.get('failed', 0),
        'excel_file': COMMON_MASTER_FILE if 'COMMON_MASTER_FILE' in locals() else None,
        'sheet': 'Department And Desgination Mast'
    })

if 'result_ct' in locals():
    summary_data.append({
        'module': 'Complaint Types',
        'created': result_ct.get('created', 0),
        'exists': result_ct.get('exists', 0),
        'failed': result_ct.get('failed', 0),
        'excel_file': COMMON_MASTER_FILE if 'COMMON_MASTER_FILE' in locals() else None,
        'sheet': 'Complaint Type Master'
    })

# Calculate totals
total_created = sum(item['created'] for item in summary_data)
total_exists = sum(item['exists'] for item in summary_data)
total_failed = sum(item['failed'] for item in summary_data)
total_records = total_created + total_exists + total_failed

# Build HTML table
table_rows = []
for item in summary_data:
    total_for_module = item['created'] + item['exists'] + item['failed']
    table_rows.append(f"""
        <tr>
            <td style="padding: 8px; border: 1px solid #ddd;">{item['module']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: green; font-weight: bold;">{item['created']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: orange; font-weight: bold;">{item['exists']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: red; font-weight: bold;">{item['failed']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; font-weight: bold;">{total_for_module}</td>
        </tr>
    """)

timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Build download links section for updated Excel files
updated_files = {}
for item in summary_data:
    if item['excel_file'] and os.path.exists(item['excel_file']):
        file_path = item['excel_file']
        file_name = os.path.basename(file_path)
        if file_path not in updated_files:
            updated_files[file_path] = {
                'name': file_name,
                'sheets': [],
                'modules': [],
                'has_errors': False,
                'has_success': False
            }
        updated_files[file_path]['sheets'].append(item['sheet'])
        updated_files[file_path]['modules'].append(item['module'])
        if item['failed'] > 0:
            updated_files[file_path]['has_errors'] = True
        if item['created'] > 0 or item['exists'] > 0:
            updated_files[file_path]['has_success'] = True

# Generate download links HTML
download_links_html = ""

# Check for legacy error file (from old approach)
legacy_error_file = 'errors/FAILED_RECORDS.xlsx'
has_legacy_errors = os.path.exists(legacy_error_file)

if updated_files:
    download_links_html = """
    <div style="margin: 20px 0; padding: 15px; background-color: #e7f3ff; border-left: 4px solid #007bff; border-radius: 5px;">
        <h3 style="margin-top: 0; color: #004085;">📥 Updated Excel Files with Status Columns</h3>
        <p style="color: #004085; margin-bottom: 15px;">
            The following Excel files have been updated with <b>_STATUS</b>, <b>_STATUS_CODE</b>, and <b>_ERROR_MESSAGE</b> columns:
        </p>
    """
    
    for file_path, file_info in updated_files.items():
        # Determine status badge
        if file_info['has_errors']:
            status_badge = '<span style="background-color: #dc3545; color: white; padding: 3px 8px; border-radius: 3px; font-size: 11px;">⚠️ HAS ERRORS</span>'
        else:
            status_badge = '<span style="background-color: #28a745; color: white; padding: 3px 8px; border-radius: 3px; font-size: 11px;">✅ ALL SUCCESS</span>'
        
        modules_list = ', '.join(file_info['modules'])
        
        download_links_html += f"""
        <div style="margin-bottom: 15px; padding: 10px; background-color: white; border-radius: 5px; border: 1px solid #ddd;">
            <div style="display: flex; justify-content: space-between; align-items: center; flex-wrap: wrap;">
                <div style="flex: 1; min-width: 300px;">
                    <strong style="color: #007bff;">📄 {file_info['name']}</strong> {status_badge}
                    <br>
                    <span style="font-size: 12px; color: #666;">Modules: {modules_list}</span>
                </div>
                <a href="{file_path}" download="{file_info['name']}" 
                   style="display: inline-block; padding: 8px 16px; background-color: #007bff; color: white; 
                          text-decoration: none; border-radius: 5px; font-weight: bold; font-size: 14px; margin-top: 5px;">
                    ⬇️ Download Updated File
                </a>
            </div>
        </div>
        """
    
    download_links_html += """
        <p style="color: #004085; font-size: 12px; margin-top: 15px; border-top: 1px solid #bee5eb; padding-top: 10px;">
            <b>💡 How to use:</b>
        </p>
        <ul style="font-size: 12px; color: #004085; margin: 5px 0;">
            <li><span style="color: green; font-weight: bold;">🟢 GREEN rows (SUCCESS):</span> Successfully created in MDMS</li>
            <li><span style="color: orange; font-weight: bold;">🟡 YELLOW rows (EXISTS):</span> Already exist in system (duplicates)</li>
            <li><span style="color: red; font-weight: bold;">🔴 RED rows (FAILED):</span> Failed to create - check _ERROR_MESSAGE column for details</li>
        </ul>
        <p style="font-size: 12px; color: #004085; margin-top: 10px;">
            <b>🔄 To retry failed records:</b> Fix the errors, delete the status columns (_STATUS, _STATUS_CODE, _ERROR_MESSAGE), and re-upload the file.
        </p>
    </div>
    """

# Add legacy error file section if it exists
if has_legacy_errors and total_failed > 0:
    download_links_html += f"""
    <div style="margin: 20px 0; padding: 15px; background-color: #fff3cd; border-left: 4px solid #ffc107; border-radius: 5px;">
        <h3 style="margin-top: 0; color: #856404;">⚠️ Legacy Error File Detected</h3>
        <p style="color: #856404; margin-bottom: 10px;">
            An error file was generated from a previous upload approach. You can download it below:
        </p>
        <div style="padding: 10px; background-color: white; border-radius: 5px; border: 1px solid #ddd;">
            <div style="display: flex; justify-content: space-between; align-items: center;">
                <div>
                    <strong style="color: #856404;">📄 FAILED_RECORDS.xlsx</strong>
                    <br>
                    <span style="font-size: 12px; color: #666;">Contains {total_failed} failed record(s)</span>
                </div>
                <a href="{legacy_error_file}" download="FAILED_RECORDS.xlsx" 
                   style="display: inline-block; padding: 8px 16px; background-color: #dc3545; color: white; 
                          text-decoration: none; border-radius: 5px; font-weight: bold; font-size: 14px;">
                    ⬇️ Download Error File
                </a>
            </div>
        </div>
        <p style="color: #856404; font-size: 12px; margin-top: 10px;">
            <b>💡 Tip:</b> Use the updated Excel files above instead - they have status columns directly in your original templates!
        </p>
    </div>
    """

html_content = f"""
<div style="font-family: Arial, sans-serif; padding: 20px; border: 2px solid #007bff; border-radius: 10px; background-color: #f8f9fa;">
    <h2 style="color: #007bff; margin-top: 0;">📊 Data Upload Summary Report</h2>
    <p style="color: #666; margin-bottom: 20px;">Generated: {timestamp}</p>
    
    <table style="width: 100%; border-collapse: collapse; margin-bottom: 20px; background-color: white;">
        <thead>
            <tr style="background-color: #007bff; color: white;">
                <th style="padding: 12px; border: 1px solid #ddd; text-align: left;">Module</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">✅ Created</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">⚠️ Already Exists</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">❌ Failed</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">📊 Total</th>
            </tr>
        </thead>
        <tbody>
            {''.join(table_rows)}
            <tr style="background-color: #e9ecef; font-weight: bold;">
                <td style="padding: 12px; border: 1px solid #ddd;">TOTAL</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: green;">{total_created}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: orange;">{total_exists}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: red;">{total_failed}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center;">{total_records}</td>
            </tr>
        </tbody>
    </table>
    
    <div style="display: flex; justify-content: space-around; margin: 20px 0; flex-wrap: wrap;">
        <div style="text-align: center; padding: 15px; background-color: #d4edda; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
            <div style="font-size: 32px; font-weight: bold; color: #155724;">{total_created}</div>
            <div style="color: #155724;">Created</div>
        </div>
        <div style="text-align: center; padding: 15px; background-color: #fff3cd; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
            <div style="font-size: 32px; font-weight: bold; color: #856404;">{total_exists}</div>
            <div style="color: #856404;">Already Exists</div>
        </div>
        <div style="text-align: center; padding: 15px; background-color: #f8d7da; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
            <div style="font-size: 32px; font-weight: bold; color: #721c24;">{total_failed}</div>
            <div style="color: #721c24;">Failed</div>
        </div>
    </div>
    
    {download_links_html}
</div>
"""

display(HTML(html_content))

print("\n🎉 DATA UPLOAD COMPLETED!")
if updated_files:
    print(f"\n📥 {len(updated_files)} Excel file(s) updated with status columns")
    for file_path in updated_files.keys():
        print(f"   • {os.path.basename(file_path)}")